In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/playground-series-s5e8/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e8/test.csv')

In [ ]:
train

train df observations:
- shape : (750000, 18)
- nulls: 0 for all columns
- -1 as pdays means they have never been contacted before

In [ ]:
#some feature engineering
train['was_contacted'] = (train['pdays'] != -1).astype(int) #a column to check if a specific customer was ever contacted or not
test['was_contacted'] = (test['pdays'] != -1).astype(int)

In [ ]:
categorical_columns = ['job', 'marital', 'education', 'contact', 'month', 'poutcome']
binary_columns = ['default', 'housing', 'loan', 'was_contacted']
num_columns = []

# **NOTE TO SELF: WHEN PREPROCESSING BINARY COLUMNS, DO NOT USE ENCODERS, BUT RATHER JUST TRANSFORM TO 0 & 1 AND TREAT AS NUMERICAL# **

In [ ]:
#preprocessing categorical columns
from sklearn.preprocessing import OneHotEncoder

def preprocess_categories(df):
    encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
    transformed = encoder.fit_transform(df[categorical_columns])
    
    encoded_df = pd.DataFrame(transformed, 
                              columns=encoder.get_feature_names_out(categorical_columns),
                              index=df.index)
    
    df = df.drop(columns=categorical_columns)
    
    df = pd.concat([df, encoded_df], axis=1)
    
    return df

train = preprocess_categories(train)

In [ ]:
#preprocessing binary columns

def yes_or_no(binary_series):
    return binary_series.apply(lambda x: 1 if x == 'yes' else 0 if x == 'no' else x)

def preprocess_binary(df):
    for col in binary_columns:
        df[col] = yes_or_no(df[col])
    return df

train = preprocess_binary(train)

In [ ]:
import lightgbm as lgb

model = lgb.LGBMClassifier()

x_train = train.drop(columns = ['id', 'y'])
y_train = train['y']

In [ ]:
model.fit(x_train, y_train)

In [ ]:
test

In [ ]:
test = preprocess_categories(test)
test = preprocess_binary(test)

In [83]:
x_test = test.drop(columns = ['id'])
y_pred = model.predict_proba(x_test)
y_pred = y_pred[:, 1]

In [84]:
submission = pd.DataFrame({
    'id' : test['id'],
    'y' : y_pred
})

In [85]:
submission.to_csv('submission.csv', index = False)